# 透過jupyter內的互動功能實際模擬使用者操作的情形

In [1]:
#匯入已加入關鍵字欄位的資料
import pandas as pd

data = pd.read_csv("heho_data_final.csv")
data

,文章標題,文章網址,圖片,部位,症狀
0,告別手腳麻木！５個超有效穴位大公開,https://heho.com.tw/archives/22330,https://heho.com.tw/wp-content/uploads/2018/09...,"上肢,下肢",養生
1,手腳冰冷怎麼辦？４穴道袪寒保暖通氣血,https://heho.com.tw/archives/151663,https://heho.com.tw/wp-content/uploads/2020/12...,"上肢,下肢",養生
2,手腕痛又麻？按摩４穴道緩解「滑鼠手」疼痛,https://heho.com.tw/archives/183035,https://heho.com.tw/wp-content/uploads/2021/07...,"上肢,手腕",滑鼠手
3,久坐易有大小毛病，按三穴道保健、解久坐不適,https://heho.com.tw/archives/192257,https://heho.com.tw/wp-content/uploads/2021/10...,下肢,久坐
4,長期久坐，水腫成蘿蔔腿！中醫傳授４穴位，小腿快速瘦一圈,https://heho.com.tw/archives/27685,https://heho.com.tw/wp-content/uploads/2018/11...,下肢,久坐
...,...,...,...,...,...
167,感冒用穴道大全，不適症狀按一按都舒緩,https://heho.com.tw/archives/27098,https://heho.com.tw/wp-content/uploads/2018/11...,NaN,感冒
168,孩子發燒感冒，按摩２穴道有助退熱止咳,https://heho.com.tw/archives/147206,https://heho.com.tw/wp-content/uploads/2020/11...,NaN,"感冒,發燒"
169,每天按4大穴道3分鐘，糖尿病就有改善!,https://heho.com.tw/archives/9177,https://heho.com.tw/wp-content/uploads/2018/03...,NaN,糖尿病
170,中醫看蕁麻疹「３觀念２穴道」舒緩止癢！,https://heho.com.tw/archives/244932,https://heho.com.tw/wp-content/uploads/2022/10...,NaN,蕁麻疹


## 使用 ipywidgets 庫創建互動元件

In [2]:
#匯入ipywidgets
#在 Jupyter 環境中創建交互式小工具。
from ipywidgets import interact
import ipywidgets as widgets

## 先讓使用者選擇要用甚麼方式做查詢

In [11]:
options=['部位', '症狀']

radio_buttons = widgets.RadioButtons(
    options=options,
    value=options[0],
    description='依_____搜尋:',
    disabled=False,
)

#'handle_radio_button_change'函式被調用並馬上打印出所選的選項
#讓使用者知道目前是選擇哪個選項

def handle_radio_button_change(change):    
    print(f"依_ {change.new}_搜尋")

radio_buttons.observe(handle_radio_button_change, 'value')

display(radio_buttons)

print("-------------------------------------------")
print()

RadioButtons(description='依_____搜尋:', options=('部位', '症狀'), value='部位')

-------------------------------------------

依_ 症狀_搜尋
依_ 部位_搜尋


## 依照使用者的選擇列出其關鍵字列表

In [8]:
import codecs
import ipywidgets as widgets
from IPython.display import Image, display


if radio_buttons.value == '症狀':
    # 讀取選項的文字檔，指定正確的編碼格式
    with codecs.open('data_syndrome.txt', 'r', encoding='utf-8') as file:
        options = [line.strip() for line in file]

    Q = widgets.SelectMultiple(
        options=options,
        value=[options[0]],
        description='症狀分類:',
        disabled=False,
    )
    #display(Q)
    column_name = '症狀'

if radio_buttons.value =='部位' :
    # 讀取選項的文字檔，指定正確的編碼格式
    with codecs.open('data_part.txt', 'r', encoding='utf-8') as file:
        options = [line.strip() for line in file]

    Q = widgets.SelectMultiple(
        options=options,
        value=[options[0]],
        description='部位分類:',
        disabled=False,
    )
    #display(Q)
    column_name = '部位'

#'handle_Q_change'函式被調用並馬上打印出所選的選項
#讓使用者知道目前是選擇哪些選項

def handle_Q_change(change):
    selected_options = ', '.join(change.new)  # 將元組轉換為字符串
    print(f"關鍵字: {selected_options} ")

Q.observe(handle_Q_change, 'value')

display(Q)

print("-------------------------------------------")
print()

SelectMultiple(description='症狀分類:', index=(0,), options=('50肩', '久坐', '中暑', '止癢', '牙齒痛', '失眠', '失智', '打嗝', '甲狀…

-------------------------------------------

關鍵字: 焦慮 
關鍵字: 眼睛疲勞, 提神 


In [13]:
# 定義不想打印的欄位列表
exclude_columns = ['部位', '症狀']

# 選取特定欄位中具有特定關鍵字的列並打印出來
col = column_name  # 選取的欄位名稱

#將 Q.value 列表中的元素以 '|' 作為分隔符連接起來形成一個新的字串。
keyword = '|'.join(Q.value)  #關鍵字

# 使用遮罩操作選取特定欄位中具有特定關鍵字的列（不包含 NA 或 NaN）
mask = data[col].notna()  # 或使用 data[col].notnull()
selected_rows = data[mask & data[col].str.contains(keyword, case=False, na=False, regex=True)]

# 迭代 DataFrame 的每一列，並打印出來（排除指定的欄位）
for _, row in selected_rows.iterrows():
    # 打印除了指定的欄位以外的其他欄位
    for column, value in row.iteritems():
        if column not in exclude_columns:
            if column == '圖片':
                display(Image(url=value, width=400, height=400))
            else:
                print(f"{column}: {value}")

    print()  # 每一列之間插入空行

文章標題: 提神醒腦必備４大穴道
文章網址: https://heho.com.tw/archives/240060



文章標題: 按這5穴道，讓你精神百倍身體好
文章網址: https://heho.com.tw/archives/10470



文章標題: 居家辦公 / 上班族必備！眼睛疲勞、肩頸僵硬，按手部穴道就搞定
文章網址: https://heho.com.tw/archives/59558
